In [ ]:
sizes = [4,5,6,7,8]
thresholds = [0.6, 0.7, 0.8, 0.9]
entry_episodes = 100

In [ ]:
import wumpus, custom_player, json

results = {}

class Devnull(object):
    def write(self, *_): pass

for size in sizes:
    print(f"Starting test on world size {size}")
    thresholds_results = {}

    for threshold in thresholds:
        print(f"\tTesting threshold {threshold}")
        episodes_results = []

        print("\t\tAttempt..", end='')
        for i in range(0, entry_episodes):
            print(f"{i+1}..", end='')
            episode_result = wumpus.run_episode(wumpus.WumpusWorld.random(size = size), custom_player.CustomPlayer(truth_threshold = threshold), horizon=1000, outf=Devnull())
            episodes_results.append([episode_result['reward'], episode_result['alive']])
        print()

        thresholds_results[threshold] = episodes_results
    
    results[size] = thresholds_results

with open('threshold_test.json', 'w') as outfile: json.dump(results, outfile)

In [ ]:
import json, statistics
from operator import itemgetter

with open('threshold_test.json') as json_file: results = json.load(json_file)

mean_rewards = []
death_percs = []

for size in sizes:
    curr_means = []
    curr_percs = []

    for threshold in thresholds:
        final_scores = list(map(itemgetter(0), results[str(size)][str(threshold)]))
        survivals = list(map(itemgetter(1), results[str(size)][str(threshold)]))
        curr_means.append(statistics.mean(final_scores))
        curr_percs.append((round(1 - sum(survivals)/len(survivals), 3)))
        
    mean_rewards.append(curr_means)
    death_percs.append(curr_percs)

In [11]:
import pandas as pd

pd.DataFrame(death_percs, columns=thresholds, index=sizes)

,0.6,0.7,0.8,0.9
4,0.09,0.01,0.0,0.0
5,0.12,0.02,0.0,0.0
6,0.17,0.03,0.0,0.0
7,0.21,0.04,0.0,0.0
8,0.22,0.08,0.0,0.0


In [12]:
pd.DataFrame(mean_rewards, columns=thresholds, index=sizes)

,0.6,0.7,0.8,0.9
4,372.26,314.91,287.07,354.58
5,259.91,213.21,272.34,305.07
6,134.93,192.17,193.03,150.31
7,-63.90,155.78,113.63,113.58
8,38.31,128.57,132.48,110.23
